In [ ]:
import syft as sy
import torch as th
import numpy as np
sy.VERBOSE = False

In [3]:
duet = sy.launch_duet()

# duet = sy.launch_duet(per_scientist_per_entity_epsilon_budget=3,
#                       universal_per_entity_epsilon_budget=5,
#                       budget_database='~./duet/privacy_budgets.db')

🎤  🎸  ♪♪♪ starting duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently 
♫♫♫ > in alpha. Do not use this to protect real-world data.
♫♫♫ >
♫♫♫ > Punching through firewall to OpenGrid Network Node at network_url: 
♫♫♫ > http://ec2-18-216-8-163.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... Skipping torch.torch.Tensor.__div__ not supported in 1.4.0
Skipping torch.torch.Tensor.__floordiv__ not supported in 1.4.0
Skipping torch.torch.Tensor.__rfloordiv__ not supported in 1.4.0
Skipping torch.torch.Tensor.bitwise_and not supported in 1.4.0
Skipping torch.torch.Tensor.bitwise_and_ not supported in 1.4.0
Skipping torch.torch.Tensor.bitwise_or not supported in 1.4.0
Skipping torch.torch.Tensor.bitwise_or_ not supported in 1.4.0
Skipping torch.torch.Tensor.div not supported in 1.4.0
Skipping torch.torch.Tensor.div_ not supported in 1.4.0
Skipping torch.torch.Tensor.floor_divide not supported in 1.4.0
Skipping torch.torch.

♫♫♫ > Duet Partner's Client Id: 7b2e4c1ae26dd0d35f516db21880132c


♫♫♫ > Connecting...
♫♫♫ > ...using a running event loop...
Skipping torch.torch.Tensor.__div__ not supported in 1.4.0
Skipping torch.torch.Tensor.__floordiv__ not supported in 1.4.0
Skipping torch.torch.Tensor.__rfloordiv__ not supported in 1.4.0
Skipping torch.torch.Tensor.bitwise_and not supported in 1.4.0
Skipping torch.torch.Tensor.bitwise_and_ not supported in 1.4.0
Skipping torch.torch.Tensor.bitwise_or not supported in 1.4.0
Skipping torch.torch.Tensor.bitwise_or_ not supported in 1.4.0
Skipping torch.torch.Tensor.div not supported in 1.4.0
Skipping torch.torch.Tensor.div_ not supported in 1.4.0
Skipping torch.torch.Tensor.floor_divide not supported in 1.4.0
Skipping torch.torch.Tensor.floor_divide_ not supported in 1.4.0
Skipping torch.torch.Tensor.is_meta not supported in 1.4.0
Skipping torch.torch.Tensor.logical_and not supported in 1.4.0
Skipping torch.torch.Tensor.logical_and_ not supported in 1.4.0
Skipping torch.torch.Tensor.logical_or not supported in 1.4.0
Skipping torc

In [6]:
data = th.tensor([[1., 2.], 
                  [2., 4.], 
                  [3., 7.], 
                  [9., 12.]])
print(data.shape)
entities = ['alice', 'bob', 'charlie', 'david']
print(len(entities))
num_entities = len(entities)
duet.store

torch.Size([4, 2])
4


[]

In [ ]:
ages = th.tensor([23,52,31,16]).tag("ages").private(every_value_unique_entity=True,
                                                    every_row_unique_entity=False,
                                                    every_column_unique_entity=False,
                                                    max_value=125,
                                                    min_value=0).send(duet, searchable=True)

# ages.entities by default equals = [acc.n_entities+0,acc.n_entities+1,acc.n_entities+2,acc.n_entities+3]

heights = th.tensor([150,160,140,100]).tag("heights").private(every_value_unique_entity=True,
                                                              max_value=250, 
                                                              min_value=0, 
                                                              entities=ages.entities).send(duet, searchable=True)

In [9]:
class RDP_Accountant:
    
    def __init__(self, num_entities):
        self.data_budget = th.tensor([1.] * num_entities)
        self.global_data_budget = 10.0
        
        self.scientist_budget = th.tensor([0.1] * num_entities)
        self.global_scientist_budget = 2.0
        
        self.rdp_guarantee = th.tensor([0.] * num_entities)
        self.global_rdp_guarantee = 0.
    
    def get_current_rdp_guarantee(self):
        return tuple(rdp_guarantee, global_rdp_guarantee)
    
    def public_approx_Renyi_filter(self, max_rounds):

        # input: public approx list of chosen epsilons and deltas (per round), 
        #        public global epsilon & delta,
        #        round of computation t / T
        # individual filtering 
        # output: Continue / Halt execution
        return True
    
    def public_approx_Renyi_odometer(self, max_rounds):

        # gives a valid upper bound on the privacy loss incurred thus far
        # input: round of computation t / T
        rdp_guarantee = tuple(alpha, eps)
        return rdp_guarantee
    
    def add_gaussian_noise(self, eps, delta, l2_sens):
        return 1.0
        
    def add_laplacian_noise(self, eps, l1_sens):
        return 1.0

In [61]:
import copy

class PrivateScalar:
    
    def __init__(self, x, g, f, c, p_id):
        
        self.p_id = p_id
        self.x = x # intermediate data
        self.g = g # polynomial
        self.f = f # lower bound val
        self.c = c # upper bound class
        
    def __repr__(self):
        
        return str("<p_id: " + self.p_id 
                   + " x: " + str(self.x)
                   + " g: " + str(self.g)
#                    + " f: " + str(self.f)
#                    + " c: " + str(self.c)
                  + ">")
    
    @staticmethod
    def create_id():
        return str(random.randint(0, 1e10))
    
    def __add__(self, other):
        x = concat_nestdicts(self.x, other.x)
        f = concat_nestdicts(self.f, other.f)
        c = concat_nestdicts(self.c, other.c)
        g = self.g + other.g
        p_id = self.create_id()
        return PrivateScalar(x, g, f, c, p_id)

def concat_nestdicts(a, b):
    x = {}
    for ent, val_dict in a.items():
        x[ent] = copy.deepcopy(val_dict)
    for ent, val_dict  in b.items():
        if ent not in x:
            x[ent] = copy.deepcopy(val_dict)
        else:
            for val_id, val in val_dict.items():
                x[ent][val_id] = val
    return x
    

In [62]:
x = [1, 2, 3, 4, 5]
entities = ['bob', 'kriti', 'andrew', 'bob', 'amber']
priv_a = gen_privscal(x[0], entities[0])
priv_b = gen_privscal(x[3], entities[3])
print(priv_a + priv_b)
print(priv_a)
print(priv_b)

<p_id: 9959622508 x: {'bob': {'bob2442327234': 1, 'bob7723779859': 4}} g: bob2442327234 + bob7723779859>
<p_id: bob2442327234 x: {'bob': {'bob2442327234': 1}} g: bob2442327234>
<p_id: bob7723779859 x: {'bob': {'bob7723779859': 4}} g: bob7723779859>


In [51]:
import random
def gen_privscal(value, entity, range_min=0., range_max=10.):

    p_id = str(entity) + str(PrivateScalar.create_id())
    x = {entity:{p_id:value}}
    g = symbols(p_id)
    f = {entity:{p_id:range_min}} 
    c = {entity:{p_id:range_max}}
    return PrivateScalar(x, g, f, c, p_id)

In [ ]:
x = [1, 2, 3, 4]
y = [2, 4, 6, 7]
ent_x = [bob, andrew, amber, bob]
y = sum(x) + x[0]
y = 2 * bob[0] + andrew[0] + amber[0] + bob[1]


In [15]:
!pip install sympy

     |████████████████████████████████| 5.8 MB 1.4 MB/s eta 0:00:01     |████████                        | 1.4 MB 1.3 MB/s eta 0:00:04
     |████████████████████████████████| 512 kB 5.5 MB/s eta 0:00:01
  Created wheel for mpmath: filename=mpmath-1.1.0-py3-none-any.whl size=532239 sha256=ffe297a438afc8271049a980f2f7e308a6102f8c57b0d3619a0824d73c95cf24
  Stored in directory: /home/kritika/.cache/pip/wheels/e2/46/78/e78f76c356bca9277368f1f97a31b37a8cb937176d9511af31
Successfully built mpmath


In [22]:
# from sympy import symbols
x, y = symbols('x y')
print(x)
print(y)
print(y * (x * 2))

x
y
2*x*y


In [29]:
expr = x + y
# expr = expr.subs(x, 0)
# expr = expr.subs(y, 1)
expr.evalf(subs={x: 2.4, y:0})
# expr
# print(expr)

2.40000000000000

In [26]:
        


    
    
    
        
# class PrivateTensor:
    
#     # each row is a data point, number of data points = no. of columns. 2-D tensor
#     def __init__(self, tensor, min_tensor, max_tensor, entity_cont, num_entities):
#         self.data = tensor
#         self.entity_cont = entity_cont # entity contribution to each data point 
#         # self.data[i] = sum over all j entity_cont[i][j]
#         self.data_min = min_tensor
#         self.data_max = max_tensor
#         self.lipschitz = []
#         self.l2_norm = []

x + 1

In [ ]:

# duet.budget

print("OVERALL - Per Scientist Entity Budget:" + str(3))
print("OVERALL - Universal Entity Budget:" + str(5))
print()
print("EST. REMAINING - MIN Universal Entity Budget:" + str(4.2))
print("EST. REMAINING - MAX Universal Entity Budget:" + str(4.9))
print("EST. REMAINING - MEAN Universal Entity Budget:" + str(4.21))
print()
print("EST. REMAINING - MIN Per Scientist Entity Budget:" + str(2.91))
print("EST. REMAINING - MAX Per Scientist Entity Budget:" + str(3.0))
print("EST. REMAINING - MEAN Per Scientist Entity Budget:" + str(2.92))

# estimated budget spend for this tensor
print("EST SPEND - MIN Per Scientist Entity Budget:" + str(0))
print("EST SPEND - MAX Per Scientist Entity Budget:" + str(0))
print("EST SPEND - MEAN Per Scientist Entity Budget:" + str(0.12))

# show analytics again as percentage ?